In [1]:
from google_drive_downloader import GoogleDriveDownloader as gdd
    
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    %cd /content
    !rm -r /content/chilung-NCTU_Adv_DNN_HW2_2
    !rm /content/test.zip
    !rm -r /content/test
    !git clone https://github.com/chilung/chilung-NCTU_Adv_DNN_HW2_2.git
    !mv /content/chilung-NCTU_Adv_DNN_HW2_2/* .
    !rm -r /content/chilung-NCTU_Adv_DNN_HW2_2
    gdd.download_file_from_google_drive(file_id='1nswVLQSGupsRympzb3tUv3L94d7ADPmi',
                dest_path='./test.zip',
                unzip=True)

Running on CoLab
/content
rm: cannot remove '/content/chilung-NCTU_Adv_DNN_HW2_2': No such file or directory
rm: cannot remove '/content/test.zip': No such file or directory
rm: cannot remove '/content/test': No such file or directory
Cloning into 'chilung-NCTU_Adv_DNN_HW2_2'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
Unzipping...Done.


In [ ]:
!ls

In [ ]:
from google.colab import drive
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    %cd /content
    !rm -r /content/test
    !rm /content/test.zip
    !git clone https://github.com/chilung/chilung-NCTU_Adv_DNN_HW2_2.git
    !mv /content/chilung-NCTU_Adv_DNN_HW2_2 .
    drive.mount('/content/drive/')
    !cp -r '/content/drive/MyDrive/Colab Notebooks/NCTU_Adv_DNN_HW2/'* /content
    !unzip /content/exec_env/test.zip >/dev/null
else:
    print('Not running on CoLab')

Running on CoLab
/content
rm: cannot remove '/content/test.zip': No such file or directory
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# -*- coding: utf-8 -*-
'''
@Time          : 20/04/25 15:49
@Author        : huguanghao
@File          : demo.py
@Noice         :
@Modificattion :
    @Author    :
    @Time      :
    @Detail    :
'''

import os
from tool.utils import *
from tool.torch_utils import *
from tool.darknet2pytorch import Darknet
import argparse
import cv2
import time
import datetime
import json
import numpy as np

"""hyper parameters"""
use_cuda = True

def output_transform(img_width, img_height, rel_center_x, rel_center_y, rel_width, rel_height):
    center_x = rel_center_x * img_width
    center_y = rel_center_y * img_height
    width = rel_width * img_width
    height = rel_height * img_height
    return (int(round(center_y-height/2)), int(round(center_x-width/2)), int(round(center_y+height/2)), int(round(center_x+width/2)))

def detect_cv2(cfgfile, weightfile, imgfiles, namesfile):
    CENTER_X = 0
    CENTER_Y = 1
    WIDTH = 2
    HEIGHT = 3
    CONFIDENCE = 4
    CLASS_ID = 6
    
    m = Darknet(cfgfile)

    # m.print_network()
    m.load_weights(weightfile)
    print('Loading weights from %s... Done!' % (weightfile))

    if use_cuda:
        m.cuda()

    num_classes = m.num_classes
    class_names = load_class_names(namesfile)

    f = open(imgfiles, 'r')
    filelist = f.read().splitlines()
    f.close()
    # print(filelist)
    
    print(time.ctime())
    start_time = time.time()

    submit_results = []
    for idx, imgfile in enumerate(filelist):
        # print(imgfile)
        img = cv2.imread(imgfile)
        sized = cv2.resize(img, (m.width, m.height))
        sized = cv2.cvtColor(sized, cv2.COLOR_BGR2RGB)
        img_height, img_width, _ = img.shape

        predict = do_detect(m, sized, 0.2, 0.6, use_cuda)[0]
        if idx%100 == 0:
            print('predict img proceed: {}'.format(idx))

        result = {}
        result['bbox'] = [
            output_transform(
                img_width, img_height,
                box[CENTER_X], box[CENTER_Y],
                box[WIDTH], box[HEIGHT])
            for box in predict]
        # print(result['bbox'])
        result['score'] = [float(box[CONFIDENCE]) for box in predict]
        # print(result['score'])
        result['label'] = [int(box[CLASS_ID]) if not box[CLASS_ID]==0 else 10 for box in predict]
        # print(result['label'])
        # print(result)
        submit_results.append(result)

    end_time = time.time()
    print(time.ctime())
    print(end_time, start_time)
    print('number of predicts: {}, fps: {}'.format(len(submit_results), len(submit_results)/(end_time-start_time)))

    print('output results for submission: {}'.format('submission.json'))
    with open('submission.json', 'w') as outfile:
       json.dump(submit_results, outfile)

    # plot_boxes_cv2(img, boxes[0], savename='predictions.jpg', class_names=class_names)

def get_args():
    parser = argparse.ArgumentParser('Test your image or video by trained model.')
    parser.add_argument('-cfgfile', type=str, default='./exec_env/yolo-obj.cfg',
                        help='path of cfg file', dest='cfgfile')
    parser.add_argument('-weightfile', type=str,
                        default='./exec_env/yolo-obj-mAP_93.3.weights',
                        help='path of trained model.', dest='weightfile')
    parser.add_argument('-imgfiles', type=str,
                        default='./exec_env/valid.txt',
                        help='path of your image file directory.', dest='imgfiles')
    parser.add_argument('-namesfile', type=str,
                        default='./exec_env/obj.names',
                        help='path of your name file.', dest='namesfile')
    args = parser.parse_args(args=['-cfgfile', './exec_env/yolo-obj.cfg',
                    '-weightfile', './exec_env/yolo-obj-mAP_93.3.weights',
                    '-imgfiles', './exec_env/valid.txt',
                    '-namesfile', './exec_env/obj.names'])
    print(args)
    # args = parser.parse_args()

    return args

if __name__ == '__main__':
    args = get_args()
    if args.imgfiles:
        detect_cv2(args.cfgfile, args.weightfile, args.imgfiles, args.namesfile)


Namespace(cfgfile='./exec_env/yolo-obj.cfg', imgfiles='./exec_env/valid.txt', namesfile='./exec_env/obj.names', weightfile='./exec_env/yolo-obj-mAP_93.3.weights')
convalution havn't activate linear
convalution havn't activate linear
convalution havn't activate linear
Loading weights from ./exec_env/yolo-obj-mAP_93.3.weights... Done!
Fri Nov 20 18:01:15 2020
predict img proceed: 0
predict img proceed: 100
predict img proceed: 200
predict img proceed: 300
predict img proceed: 400
predict img proceed: 500
predict img proceed: 600
predict img proceed: 700
predict img proceed: 800
predict img proceed: 900
predict img proceed: 1000
predict img proceed: 1100
predict img proceed: 1200
predict img proceed: 1300
predict img proceed: 1400
predict img proceed: 1500
predict img proceed: 1600
predict img proceed: 1700
predict img proceed: 1800
predict img proceed: 1900
predict img proceed: 2000
predict img proceed: 2100
predict img proceed: 2200
predict img proceed: 2300
predict img proceed: 2400
pr

In [ ]:
!cp submission.json '/content/drive/MyDrive/Colab Notebooks/NCTU_Adv_DNN_HW2/'